In [ ]:
import numpy as np

import matplotlib.pyplot as plt

import pybullet

import time
plt.ion()
img = np.random.rand(200, 320)

#img = [tandard_normal((50,100))

image = plt.imshow(img,interpolation='none',animated=True,label="blah")

ax = plt.gca()
#pybullet.connect(pybullet.GUI)

#pybullet.connect(pybullet.DIRECT)
pybullet.connect(pybullet.SHARED_MEMORY)


pybullet.loadURDF("plane.urdf",[0,0,-1])

#pybullet.loadURDF("r2d2.urdf")
pybullet.loadURDF("D:/bullet3_vr/examples/pybullet/gym/pybullet_data/baxter_common/baxter_description/urdf/toms_baxter.urdf")


camTargetPos = [0,0,0]

cameraUp = [0,0,1]

cameraPos = [1,1,1]

pybullet.setGravity(0,0,-10)



pitch = -10.0



roll=0

upAxisIndex = 2

camDistance = 4

pixelWidth = 120

pixelHeight = 120

nearPlane = 0.01

farPlane = 100

fov = 60


In [ ]:
yaw = 20
viewMatrix = pybullet.computeViewMatrixFromYawPitchRoll(camTargetPos, camDistance, yaw, pitch, roll, upAxisIndex)
aspect = pixelWidth / pixelHeight;

projectionMatrix = pybullet.computeProjectionMatrixFOV(fov, aspect, nearPlane, farPlane);

In [ ]:
img_arr = pybullet.getCameraImage(pixelWidth, pixelHeight,
   viewMatrix,projectionMatrix, renderer=pybullet.ER_BULLET_HARDWARE_OPENGL)
#shadow=1,lightDirection=[1,1,1],ER_TINY_RENDERER  ER_BULLET_HARDWARE_OPENGL

In [ ]:


w=img_arr[0] #width of the image, in pixels
h = img_arr[1]
print(w)
rgb=img_arr[2] #color data RGB
#print(len(rgb))
np_img_arr = np.reshape(rgb, (h, w, 4))
np_img_arr = np_img_arr*(1./255.)
print(len(img_arr[3]))
 depth_buffer_opengl =np.reshape(img_arr[3], [h, w])
print(len(depth_buffer_opengl))
depth_opengl = farPlane * nearPlane / (farPlane - (farPlane - nearPlane) * depth_buffer_opengl)
print(depth_opengl[0])
seg_opengl = np.reshape(img_arr[4], [w, h])*1./255.

#test        
image = plt.imshow(np_img_arr,interpolation='none',animated=True,label="blah")



image.set_data(np_img_arr)

ax.plot([0])

    #plt.draw()

plt.show()

In [ ]:
image_depth = plt.imshow(depth_opengl,interpolation='none',animated=True,label="blah")

image_depth.set_data(depth_opengl)

ax.plot([0])

    #plt.draw()

plt.show()

In [41]:
depth_opengl=np.array(img_arr[3])
print(depth_opengl)

print(depth_opengl.dtype)
depth_opengl_conv = farPlane * nearPlane / (farPlane - (farPlane - nearPlane) * depth_opengl)
print(depth_opengl_conv)

print(depth_opengl_conv.dtype)
fp_d = open('depthFile' + '.bin', 'wb')


[1.         1.         1.         ... 0.99574769 0.99574769 0.99574769]
float64
[100.         100.         100.         ...   2.29785255   2.29785255
   2.29785255]
float64


In [ ]:
depth_opengl=np.array(img_arr[3])
print(depth_opengl)

print(depth_opengl.dtype)







#depthImageRecorder.add_image(ts,depth_opengl) 

import cv2
#import cv_bridge
import numpy as np

import snappy
import struct


print(len(depth_opengl))
if depth_opengl.dtype == np.float32:
            # Is simulated data
            mask = np.isnan(depth_opengl)
            print(mask)
            if mask.any():
                # In simulation, the background has NaN depth values.
                # We replace them with 0 m, similar to what the Kinect V1 did.
                # See https://msdn.microsoft.com/en-us/library/jj131028.aspx.
                #rospy.logdebug("There was at least one NaN in the depth image. " +
                #              "I replaced all occurrences with 0.0 m.")
                depth_opengl.flags.writeable = True
                depth_opengl[mask] = 0.0
                # We now map the float values in meters to uint16 values in mm
                # as provided by the libfreenect2 library and Kinect SDK.
                depth_opengl *= 1000.0
                depth_opengl = depth_opengl.astype(np.uint16, copy=False)
                assert depth_opengl.dtype == np.uint16

img_comp = snappy.compress(depth_opengl)
print(len(img_comp))
        # write number of bytes of compressed image
nr_bytes = struct.pack('<L', len(img_comp))
fp_d.write(nr_bytes)
        # write compressed image
fp_d.write(img_comp)
fp_d.flush()


In [ ]:
#decompress with snappy
imgsize = (120,120)
images = list()
with open('depthFile.bin', 'rb') as fp:
            b = fp.read(4)
            
            #while b != '':
            while len(b) >= 4:
                k = struct.unpack('<L', b)[0]
                print(k)
                image_bytes = fp.read(k)
                images.append(snappy.uncompress(image_bytes))
                b = fp.read(4)
l = len(images)
#print(l)
images = np.array(images)
#images = np.fromstring(images, dtype=np.dtype('>u2'))
images = np.frombuffer(images, dtype=np.float64)
images=images.reshape((l,) + (-1,))
#depth_img_arr=np.reshape(images, (h, w, 4))
#depthImage recorder test
#images=np.array(images)
#print(len(images))
#print(images)

In [ ]:
# how to work with decompressed image:
for i in range (0,len(images),1):
    depth_buffer_opengl =np.reshape(images[i,:], [120, 120])
    #print(len(depth_buffer_opengl))
    depth_opengl = farPlane * nearPlane / (farPlane - (farPlane - nearPlane) * depth_buffer_opengl)
    image_depth = plt.imshow(depth_opengl,interpolation='none',animated=True,label="blah")
    image_depth.set_data(depth_opengl)
    ax.plot([0])

    #plt.draw()
    plt.show()

In [ ]:
depth_buffer_opengl =np.reshape(images[1,:], [120, 120])
print(len(depth_buffer_opengl))
depth_opengl = farPlane * nearPlane / (farPlane - (farPlane - nearPlane) * depth_buffer_opengl)
image_depth = plt.imshow(depth_opengl,interpolation='none',animated=True,label="blah")

image_depth.set_data(depth_opengl)

ax.plot([0])

    #plt.draw()

plt.show()

In [ ]:
#depthImageRecorder.add_image(ts,depth_opengl) 

import cv2
#import cv_bridge
import numpy as np

import snappy
import struct

fp_d = open('depthFile' + '.bin', 'wb')

print(len(img_arr[3]))
if img_arr[3].dtype == np.float32:
            # Is simulated data
            mask = np.isnan(img_arr[3])
            print(mask)
            if mask.any():
                # In simulation, the background has NaN depth values.
                # We replace them with 0 m, similar to what the Kinect V1 did.
                # See https://msdn.microsoft.com/en-us/library/jj131028.aspx.
                #rospy.logdebug("There was at least one NaN in the depth image. " +
                #              "I replaced all occurrences with 0.0 m.")
                img_arr[3].flags.writeable = True
                img_arr[3][mask] = 0.0
                # We now map the float values in meters to uint16 values in mm
                # as provided by the libfreenect2 library and Kinect SDK.
                img_arr[3] *= 1000.0
                img_arr[3] = img_arr[3].astype(np.uint16, copy=False)
                assert img_arr[3].dtype == np.uint16

img_comp = snappy.compress(depth_opengl)
print(len(img_comp))
        # write number of bytes of compressed image
nr_bytes = struct.pack('<L', len(img_comp))
fp_d.write(nr_bytes)
        # write compressed image
fp_d.write(img_comp)
fp_d.flush()
imgsize = (120,120)
images = list()
with open('depthFile.bin', 'rb') as fp:
            b = fp.read(4)
            
            #while b != '':
            while len(b) >= 4:
                k = struct.unpack('<L', b)[0]
                print(k)
                image_bytes = fp.read(k)
                images.append(snappy.uncompress(image_bytes))
                b = fp.read(4)
l = len(images)
print(l)
images = np.array(images)
#images = np.fromstring(images, dtype=np.dtype('>u2'))
images = np.frombuffer(images, dtype=np.dtype('>u2'))
#images.reshape((-1,) + imgsize)
#depth_img_arr=np.reshape(images, (h, w, 4))
#depthImage recorder test
#images=np.array(images)
print(len(images))
print(images)

In [ ]:
image_seg= plt.imshow(images,interpolation='none',animated=True,label="blah")
image_seg.set_data(images)

ax.plot([0])

    #plt.draw()

plt.show()

In [ ]:
image_seg= plt.imshow(seg_opengl,interpolation='none',animated=True,label="blah")
image_seg.set_data(seg_opengl)

ax.plot([0])

    #plt.draw()

plt.show()

In [ ]:
main_start = time.time()
t=1
#logId = pybullet.startStateLogging(pybullet.STATE_LOGGING_VIDEO_MP4,"vidoelog")
while (t<10):

  for yaw in range (0,360,10):

    pybullet.stepSimulation()

    start = time.time()

    t=t+1

    viewMatrix = pybullet.computeViewMatrixFromYawPitchRoll(camTargetPos, camDistance, yaw, pitch, roll, upAxisIndex)

    aspect = pixelWidth / pixelHeight;

    projectionMatrix = pybullet.computeProjectionMatrixFOV(fov, aspect, nearPlane, farPlane);

    img_arr = pybullet.getCameraImage(pixelWidth, pixelHeight, viewMatrix,projectionMatrix, shadow=1,lightDirection=[1,1,1],renderer=pybullet.ER_BULLET_HARDWARE_OPENGL)

    stop = time.time()

    print ("renderImage %f" % (stop - start))



    w=img_arr[0] #width of the image, in pixels

    h=img_arr[1] #height of the image, in pixels

    rgb=img_arr[2] #color data RGB

    dep=img_arr[3] #depth data



    print ('width = %d height = %d' % (w,h))



    #note that sending the data to matplotlib is really slow



    #reshape is needed

    np_img_arr = np.reshape(rgb, (h, w, 4))

    np_img_arr = np_img_arr*(1./255.)

    

    #show

    #plt.imshow(np_img_arr,interpolation='none',extent=(0,1600,0,1200))

    image = plt.imshow(np_img_arr,interpolation='none',animated=True,label="blah")



    image.set_data(np_img_arr)

    ax.plot([0])

    #plt.draw()

    plt.show()

    plt.pause(0.01)

    #image.draw()

    



main_stop = time.time()



print ("Total time %f" % (main_stop - main_start))

#pybullet.stopStateLogging(logId)

#pybullet.resetSimulation()